In [8]:
# 1. Librerías necesarias
import pandas as pd
import sqlite3
from IPython.display import display, Markdown

# 2. Cargar dataset
df = pd.read_csv("heart.csv")

# 3. Crear conexión a SQLite en memoria
conn = sqlite3.connect(":memory:")

# 4. Guardar el DataFrame como tabla SQL
df.to_sql("pacientes", conn, index=False, if_exists="replace")

# 5. Función auxiliar para ejecutar y mostrar queries con título
def run_query(query, title):
    display(Markdown(f"### 🔹 {title}"))
    result = pd.read_sql(query, conn)
    display(result)
    display(Markdown("---"))  # línea divisoria

# ============================
# 6. Queries con títulos
# ============================

# 1️⃣ Total de pacientes y porcentaje diagnosticado
q1 = """
SELECT
    COUNT(*) AS total_pacientes,
    SUM(HeartDisease) AS pacientes_con_enfermedad,
    ROUND(100.0 * SUM(HeartDisease) / COUNT(*), 2) AS porcentaje_enfermedad
FROM pacientes;
"""
run_query(q1, "1️⃣ Total de pacientes y porcentaje diagnosticado con enfermedad cardíaca")

# 2️⃣ Edad promedio y colesterol medio según condición cardíaca
q2 = """
SELECT
    HeartDisease,
    ROUND(AVG(Age), 2) AS edad_promedio,
    ROUND(AVG(Cholesterol), 2) AS colesterol_medio
FROM pacientes
GROUP BY HeartDisease;
"""
run_query(q2, "2️⃣ Edad promedio y colesterol medio por condición cardíaca")

# 3️⃣ Distribución por sexo y condición cardíaca
q3 = """
SELECT
    Sex,
    HeartDisease,
    COUNT(*) AS num_pacientes
FROM pacientes
GROUP BY Sex, HeartDisease
ORDER BY Sex, HeartDisease;
"""
run_query(q3, "3️⃣ Distribución de pacientes por sexo y condición cardíaca")

# 4️⃣ Segmentación por tipo de dolor de pecho
q4 = """
SELECT
    ChestPainType,
    COUNT(*) AS total_pacientes,
    SUM(HeartDisease) AS casos_enfermedad,
    ROUND(100.0 * SUM(HeartDisease) / COUNT(*), 2) AS tasa_enfermedad_pct
FROM pacientes
GROUP BY ChestPainType
ORDER BY tasa_enfermedad_pct DESC;
"""
run_query(q4, "4️⃣ Segmentación por tipo de dolor de pecho y tasa de enfermedad")

# 5️⃣ Pacientes de alto riesgo
q5 = """
SELECT
    Age,
    Sex,
    ExerciseAngina,
    MaxHR,
    Cholesterol
FROM pacientes
WHERE Age > 55
  AND ExerciseAngina = 'Y'
  AND HeartDisease = 1;
"""
run_query(q5, "5️⃣ Pacientes de alto riesgo (mayores de 55 años con angina inducida por ejercicio)")

### 🔹 1️⃣ Total de pacientes y porcentaje diagnosticado con enfermedad cardíaca

,total_pacientes,pacientes_con_enfermedad,porcentaje_enfermedad
0,918,508,55.34


---

### 🔹 2️⃣ Edad promedio y colesterol medio por condición cardíaca

,HeartDisease,edad_promedio,colesterol_medio
0,0,50.55,227.12
1,1,55.90,175.94


...

,ChestPainType,total_pacientes,casos_enfermedad,tasa_enfermedad_pct
0,ASY,496,392,79.03
1,TA,46,20,43.48
2,NAP,203,72,35.47
3,ATA,173,24,13.87


---

### 🔹 5️⃣ Pacientes de alto riesgo (mayores de 55 años con angina inducida por ejercicio)

,Age,Sex,ExerciseAngina,MaxHR,Cholesterol
0,58,M,Y,99,164
1,65,M,Y,87,306
2,57,M,Y,145,265
3,56,M,Y,125,213
4,66,M,Y,94,139
...,...,...,...,...,...
179,58,F,Y,146,225
180,63,M,Y,144,187
181,63,F,Y,136,197
182,57,F,Y,123,241


---